In [1]:
!python -m venv myenv


In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 36.4 MB 4.8 MB/s            
     |████████████████████████████████| 40.1 MB 101.0 MB/s            


In [2]:
!aws s3 ls commoncrawl/

                           PRE .test/
                           PRE cc-index/
                           PRE contrib/
                           PRE crawl-001/
                           PRE crawl-002/
                           PRE crawl-analysis/
                           PRE crawl-data/
                           PRE data-jsonl/
                           PRE hive_analysis/
                           PRE index2012/
                           PRE mapred-temp/
                           PRE meanpath/
                           PRE parse-output-test/
                           PRE parse-output/
                           PRE projects/
                           PRE static/
                           PRE stats-output/
                           PRE wikipedia/
2024-06-01 08:15:19       1366 index.html
2024-03-24 22:59:18       5384 privacy.txt
2016-05-21 18:17:43         25 robots.txt
2024-03-08 17:45:23      29480 termsofuse.txt


In [3]:
import duckdb
import pandas as pd

In [4]:
!aws s3 cp s3://crawler-works/cc_index.duckdb .

download: s3://crawler-works/cc_index.duckdb to ./cc_index.duckdb    


In [5]:
con = duckdb.connect('cc_index.duckdb')
con.install_extension("aws")
con.load_extension("aws")

con.install_extension("s3")
con.load_extension("s3")

con.install_extension("httpfs")
con.load_extension("httpfs")

In [6]:
CRAWL_SET = "CC-MAIN-2024-42"
CC_URL_S3_BASE="s3://"
CC_URL_HTTP_BASE="https://data.commoncrawl.org/"
CC_INDEX_URL = f"{CC_URL_HTTP_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_BASE_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/"
CC_INDEX_URL

'https://data.commoncrawl.org/commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [7]:
CC_INDEX_S3_URL

's3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [8]:
print(f"{CC_INDEX_S3_BASE_URL}")

s3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/


In [9]:
# Configure S3 settings for DuckDB
con.execute("""
    CALL load_aws_credentials();
""")

In [10]:
con.execute("""
  DROP SECRET IF EXISTS secret3;
  CREATE SECRET secret3 (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    CHAIN 'sso'
  );
            """)

--con.execute(f"""
  CREATE TABLE IF NOT EXISTS t1 as 
  SELECT *
    FROM read_parquet('{CC_INDEX_S3_URL}');
  DESCRIBE t1;
""").df()

In [11]:
!ls

LICENSE		 examples  jupyter-server  requirements.txt   warc-rs
cc_index.duckdb  fixed-ip  myenv	   terraform.tfstate
cdx.db		 hello.db  pipeline.ipynb  testdb


In [12]:
con.execute('select count(*) from t1').df()

,count_star()
0,2495190754


In [13]:
con.execute('describe t1').df()

,column_name,column_type,null,key,default,extra
0,url_surtkey,VARCHAR,YES,None,None,None
1,url,VARCHAR,YES,None,None,None
2,url_host_name,VARCHAR,YES,None,None,None
3,url_host_tld,VARCHAR,YES,None,None,None
4,url_host_2nd_last_part,VARCHAR,YES,None,None,None
5,url_host_3rd_last_part,VARCHAR,YES,None,None,None
6,url_host_4th_last_part,VARCHAR,YES,None,None,None
7,url_host_5th_last_part,VARCHAR,YES,None,None,None
8,url_host_registry_suffix,VARCHAR,YES,None,None,None
9,url_host_registered_domain,VARCHAR,YES,None,None,None


In [14]:
con.execute('select * from t1 limit 3').df().transpose() 

,0,1,2
url_surtkey,"com,blogspot,blogdocappacete)/2012/11/a-breve-...","com,blogspot,blogdocappacete)/2012/11/burguesi...","com,blogspot,blogdocappacete)/2012/11/deputado..."
url,https://blogdocappacete.blogspot.com/2012/11/a...,https://blogdocappacete.blogspot.com/2012/11/b...,https://blogdocappacete.blogspot.com/2012/11/d...
url_host_name,blogdocappacete.blogspot.com,blogdocappacete.blogspot.com,blogdocappacete.blogspot.com
url_host_tld,com,com,com
url_host_2nd_last_part,blogspot,blogspot,blogspot
url_host_3rd_last_part,blogdocappacete,blogdocappacete,blogdocappacete
url_host_4th_last_part,None,None,None
url_host_5th_last_part,None,None,None
url_host_registry_suffix,com,com,com
url_host_registered_domain,blogspot.com,blogspot.com,blogspot.com


In [15]:
con.execute('select count(distinct  url_host_name) from t1').df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count(DISTINCT url_host_name)
0,47550631


In [17]:
#pages per host
con.execute('select count(*)/count(distinct url_host_name) from t1').df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,(count_star() / count(DISTINCT url_host_name))
0,52.474398


In [19]:
con.execute('select count(distinct warc_filename) from t1').df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count(DISTINCT warc_filename)
0,90000


In [32]:
p1pct_db_path = 'cc_index_p1pct_sample.duckdb'
p1pct_sample_fraction = ".1%"
con.execute(f"ATTACH '{p1pct_db_path}' AS sample_db_p1pct")
con.execute(f"""
    CREATE TABLE sample_db_p1pct.cc_index_sample AS
    SELECT * FROM t1 USING SAMPLE {p1pct_sample_fraction};
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [29]:
!aws s3 cp cc_index_15pct_sample.duckdb s3://crawler-works/

upload: ./cc_index_15pct_sample.duckdb to s3://crawler-works/cc_index_15pct_sample.duckdb


In [27]:
second_db_path = 'cc_index_15pct_sample.duckdb'
con.execute(f"ATTACH '{second_db_path}' AS sample_db")

In [30]:
sample_fraction = "15%"
con.execute(f"""
    CREATE TABLE sample_db.cc_index_sample AS
    SELECT * FROM t1 USING SAMPLE {sample_fraction};
""")

In [39]:
con.execute(f"""
    select count(*) from sample_db.cc_index_sample
""").df()

,count_star()
0,375731115


In [38]:
con.execute(f"""
    select count(*) from sample_db_p1pct.cc_index_sample
""").df()

,count_star()
0,2464920


In [40]:
!aws s3 cp {p1pct_db_path} s3://crawler-works/

upload: ./cc_index_p1pct_sample.duckdb to s3://crawler-works/cc_index_p1pct_sample.duckdb


# IGNORE EVERYTHING BELOW THIS LINE

In [ ]:
!duckdb -c "\
  create table meta as SELECT * FROM '../cc/cdx-00000.jsonl'; \
  create view status_vw as Select status, count(status) as cnt from meta group by status order by cnt desc; \
  select * from meta; \
  select * from status_vw; \
" {DB_FILE}

In [ ]:
!duckdb -c ".databases" {DB_FILE}
#!duckdb -c ".open hello.db"

In [ ]:
!duckdb -c " \
create table metastats as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta group by tstatus order by cstatus desc; \
select * from metastats; \
" {DB_FILE}

In [ ]:
!duckdb -c " \
create table metastats_norobotstxt as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta where url not like '%robots.txt' group by tstatus order by cstatus desc; \
select * from metastats_norobotstxt; \
" {DB_FILE}

In [ ]:
#!duckdb -c "drop table metastats" {DB_FILE}

In [ ]:
first_line = ""
with open("../cc/cdx-00000", "r") as file:
    first_line = file.readline()

print(first_line)

In [ ]:
!echo "{first_line}" | cut -d' ' -f3- 

In [ ]:
import json
s = '{"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}'
json.loads(s)